In [1]:
import json
import pandas
import numpy as np
import pandas as pd

In [ ]:
NUM_OF_ZONE = 258
HUB_INDEXES = [131,140,180]

In [ ]:
zone_file = pd.read_csv("data/zone_fileNYC.csv")

In [20]:
fhv = []

def get_month(i):
    if i < 10:
        return '0' + str(i)
    else:
        return str(i)

for i in range(1,13):
    fhv.append(pd.read_parquet("fhv_tripdata_2019-" + get_month(i)+".parquet", engine="fastparquet",
                              columns = ['pickup_datetime', 'PUlocationID' , 'DOlocationID'] ))

In [23]:
fhv = pd.concat(fhv)

In [26]:
fhvhv = []

def get_month(i):
    if i < 10:
        return '0' + str(i)
    else:
        return str(i)

for i in range(2,13):
    fhvhv.append(pd.read_parquet("fhvhv_tripdata_2019-" + get_month(i)+".parquet", engine="fastparquet",
                              columns = ['pickup_datetime', 'PULocationID' , 'DOLocationID'] ))

In [27]:
fhvhv = pd.concat(fhvhv)

In [28]:
yellow = pd.read_csv("yellow2019_h1.csv")
yellow = yellow.append(pd.read_csv("yellow2019_h2.csv"))

C:\Users\User\AppData\Local\Temp\ipykernel_32164\2159289457.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  yellow = yellow.append(pd.read_csv("yellow2019_h2.csv"))


In [29]:
green = pd.read_csv("green2019_h1.csv")
green = green.append(pd.read_csv("green2019_h2.csv"))

C:\Users\User\AppData\Local\Temp\ipykernel_32164\4081380501.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  green = green.append(pd.read_csv("green2019_h2.csv"))


In [30]:
# Let's get the demand! 
fhvhv['day'] = fhvhv['pickup_datetime'].dt.dayofyear
fhvhv['hour'] = fhvhv['pickup_datetime'].dt.hour

In [ ]:
fhvhv = fhvhv.groupby(["day", "hour", "PULocationID", "DOLocationID"]).count().reset_index()

In [37]:
# Let's get the demand! 
fhv['day'] = fhv['pickup_datetime'].dt.dayofyear
fhv['hour'] = fhv['pickup_datetime'].dt.hour

In [38]:
fhv = fhv.groupby(["day", "hour", "PUlocationID", "DOlocationID"]).count().reset_index()

In [56]:
fhv = fhv[(fhv['PUlocationID'] <= 263) & (fhv['DOlocationID'] <= 263)].reset_index()
fhvhv = fhvhv[(fhvhv['PULocationID'] <= 263) & (fhvhv['DOLocationID'] <= 263)].reset_index()
yellow = yellow[(yellow['pickup_loc'] <= 263) & (yellow['dropoff_loc'] <= 263)].reset_index()
green = green[(green['pickup_loc'] <= 263) & (green['dropoff_loc'] <= 263)].reset_index()

In [68]:
yellow = yellow[yellow['day']<366].reset_index()
green = green[green['day']<366].reset_index()

In [70]:
# create the matrix we need
mat = np.zeros((24*365, 263, 263))

In [71]:
x = (fhv['day'].values-1) * 24 + fhv['hour'].values
y = fhv['PUlocationID'].values-1
z = fhv['DOlocationID'].values-1
value =  fhv['pickup_datetime'].values
x = x.astype(int)
y = y.astype(int)
z = z.astype(int)

mat[x,y,z] += value

In [72]:
x = (fhvhv['day'].values-1) * 24 + fhvhv['hour'].values
y = fhvhv['PULocationID'].values-1
z = fhvhv['DOLocationID'].values-1
value =  fhvhv['pickup_datetime'].values
x = x.astype(int)
y = y.astype(int)
z = z.astype(int)

mat[x,y,z] += value

In [73]:
x = (yellow['day'].values-1) * 24 + yellow['hour'].values
y = yellow['pickup_loc'].values-1
z = yellow['dropoff_loc'].values-1
value =  yellow['count'].values
x = x.astype(int)
y = y.astype(int)
z = z.astype(int)

mat[x,y,z] += value

In [74]:
x = (green['day'].values-1) * 24 + green['hour'].values
y = green['pickup_loc'].values-1
z = green['dropoff_loc'].values-1
value =  green['count'].values
x = x.astype(int)
y = y.astype(int)
z = z.astype(int)

mat[x,y,z] += value

In [ ]:
# sanity check here
np.sum(mat)/365

In [76]:
np.save("demand_2019.npy", mat)

In [3]:
mat = np.load("demand_2019.npy")

In [7]:
selected_index = zone_file['LocationID'].values

In [ ]:
mat = mat[:,selected_index-1,:][:,:,selected_index-1]

In [8]:
import os
from datetime import datetime

In [9]:
# Get the index of the scenarios and corresponding dates
for k in range(4):
    for filename in os.listdir("demand/scenario" + str(k)):
        if filename.startswith("demand"):
            day = filename.split("_")[1].split(".")[0]
            day_index = datetime.strptime(day,"%Y-%m-%d").timetuple().tm_yday - 1
            data = mat[((day_index)*24-3):((day_index+1)*24+3),:,:]
            print(np.sum(data))
            res = dict()
            for i in range(NUM_OF_ZONE):
                for j in range(NUM_OF_ZONE):
                    if(np.sum( data[:, i, j]) > 0):
                        res[str((i, j))] = data[:, i, j].tolist()
            with open("demand_full/scenario" + str(k) + "/" + filename.split(".")[0] + ".json", 'w') as f:
                json.dump(res, f)

1151031.0
1231662.0
1109966.0
1054612.0
1069206.0
1137758.0
1229058.0
993804.0
1234134.0
909685.0
1013448.0
1058354.0
1032967.0
998210.0
1061164.0
920847.0
851227.0
1083426.0
1113384.0
1095431.0
984365.0
1358848.0
1530860.0
1169415.0
1108854.0
1140617.0
987072.0
878877.0
1094086.0
1208814.0
1018628.0
1119712.0
967326.0
1284279.0
1011016.0
1020078.0
1019942.0
978960.0
1070580.0
997304.0
1057386.0
1423973.0
1261998.0
1003431.0
1263857.0
990689.0
1020568.0
1076827.0
1062060.0
1404848.0
1010668.0
1331688.0
1107228.0
901276.0
1133239.0
1145622.0
1049812.0
851068.0
1136909.0
1144127.0
918992.0
1117883.0
1108612.0
1030218.0
1095831.0
988094.0
1003285.0


In [10]:
# Get the index of the scenarios and corresponding dates
for k in range(4):
    for filename in os.listdir("demand/scenario" + str(k)):
        if filename.startswith("demand"):
            day = filename.split("_")[1].split(".")[0]
            day_index = datetime.strptime(day,"%Y-%m-%d").timetuple().tm_yday - 1
            data = mat[(day_index*24-3):((day_index+1)*24+3),:,:]
            print(np.sum(data))
            res = dict()
            for i in HUB_INDEXES:
                for j in range(NUM_OF_ZONE):
                    if(np.sum( data[:, i, j]) > 0):
                        res[str((i, j))] = data[:, i, j].tolist()
            for i in range(NUM_OF_ZONE):
                for j in HUB_INDEXES:
                    if(np.sum( data[:, i, j]) > 0):
                        res[str((i, j))] = data[:, i, j].tolist()
            with open("demand/scenario" + str(k) + "/" + filename.split(".")[0] + ".json", 'w') as f:
                json.dump(res, f)

1151031.0
1231662.0
1109966.0
1054612.0
1069206.0
1137758.0
1229058.0
993804.0
1234134.0
909685.0
1013448.0
1058354.0
1032967.0
998210.0
1061164.0
920847.0
851227.0
1083426.0
1113384.0
1095431.0
984365.0
1358848.0
1530860.0
1169415.0
1108854.0
1140617.0
987072.0
878877.0
1094086.0
1208814.0
1018628.0
1119712.0
967326.0
1284279.0
1011016.0
1020078.0
1019942.0
978960.0
1070580.0
997304.0
1057386.0
1423973.0
1261998.0
1003431.0
1263857.0
990689.0
1020568.0
1076827.0
1062060.0
1404848.0
1010668.0
1331688.0
1107228.0
901276.0
1133239.0
1145622.0
1049812.0
851068.0
1136909.0
1144127.0
918992.0
1117883.0
1108612.0
1030218.0
1095831.0
988094.0
1003285.0
